In [1]:
# Importar librerías
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier

In [2]:
# 1. Obtener un set de datos.
df = pd.read_csv("alzheimers_disease_data.csv")

# 2. Buscar valores vacios
print("Missing values:")
print(df.isna().sum())

# 3. Eliminar columnas que no agrega valor al entrenamiento
df = df.drop('PatientID', axis=1)
df = df.drop('DoctorInCharge', axis=1)

print("\nDistribución original de clases:")
print(df['Diagnosis'].value_counts(normalize=True) * 100)

Missing values:
PatientID                    0
Age                          0
Gender                       0
Ethnicity                    0
EducationLevel               0
BMI                          0
Smoking                      0
AlcoholConsumption           0
PhysicalActivity             0
DietQuality                  0
SleepQuality                 0
FamilyHistoryAlzheimers      0
CardiovascularDisease        0
Diabetes                     0
Depression                   0
HeadInjury                   0
Hypertension                 0
SystolicBP                   0
DiastolicBP                  0
CholesterolTotal             0
CholesterolLDL               0
CholesterolHDL               0
CholesterolTriglycerides     0
MMSE                         0
FunctionalAssessment         0
MemoryComplaints             0
BehavioralProblems           0
ADL                          0
Confusion                    0
Disorientation               0
PersonalityChanges           0
DifficultyCompletingTas

In [3]:
# Guardar las columnas por normalizar
numerical_cols = ["Age", "BMI", "AlcoholConsumption", "PhysicalActivity", "DietQuality", "SleepQuality", "SystolicBP", "DiastolicBP", "CholesterolTotal", "CholesterolLDL", "CholesterolHDL", "CholesterolTriglycerides", "MMSE", "FunctionalAssessment", "ADL"]

# Guardar las columnas numéricas
categorical_cols = ["Gender", "Ethnicity", "EducationLevel", "Smoking", "FamilyHistoryAlzheimers", "CardiovascularDisease", "Diabetes", "Depression", "HeadInjury", "Hypertension", "MemoryComplaints", "BehavioralProblems", "Confusion", "Disorientation", "PersonalityChanges", "DifficultyCompletingTasks", "Forgetfulness"]

In [4]:
# 4. Separar features y target
X = df.drop('Diagnosis', axis=1)
y = df['Diagnosis']

In [5]:
# 5. Dividir en train y test con un split 80 - 20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# 6. Aplicar las técnicas de escalamiento y transformación de datos
# Transformar datos categóricos mediante One Hot Encoder
encoder = OneHotEncoder(sparse_output=False)
X_train_cat_encoded = encoder.fit_transform(X_train[categorical_cols])
X_test_cat_encoded = encoder.transform(X_test[categorical_cols])

# Escalamiento de datos numéricos mediante StandarScaler
scaler = StandardScaler()
X_train_num_scaled = scaler.fit_transform(X_train[numerical_cols])
X_test_num_scaled = scaler.transform(X_test[numerical_cols])

In [7]:
# 6. Aplicar las técnicas de escalamiento y transformación de datos
# Transformar datos categóricos mediante One Hot Encoder
encoder = OneHotEncoder(sparse_output=False)
X_train_cat_encoded = encoder.fit_transform(X_train[categorical_cols])
X_test_cat_encoded = encoder.transform(X_test[categorical_cols])

# Escalamiento de datos numéricos mediante StandarScaler
scaler = StandardScaler()
X_train_num_scaled = scaler.fit_transform(X_train[numerical_cols])
X_test_num_scaled = scaler.transform(X_test[numerical_cols])

In [8]:
# 7. Obtener nombres de features
encoded_feature_names = encoder.get_feature_names_out(categorical_cols)

# Crear DataFrames para datos categóricos transformados
X_train_cat_df = pd.DataFrame(X_train_cat_encoded, columns=encoded_feature_names)
X_test_cat_df = pd.DataFrame(X_test_cat_encoded, columns=encoded_feature_names)

# Crear DataFrames para datos numéricos transformados
X_train_num_df = pd.DataFrame(X_train_num_scaled, columns=numerical_cols)
X_test_num_df = pd.DataFrame(X_test_num_scaled, columns=numerical_cols)

In [9]:
# 8. Combinar datos categóricos y numéricos
X_train_processed = pd.concat([X_train_num_df, X_train_cat_df], axis=1)
X_test_processed = pd.concat([X_test_num_df, X_test_cat_df], axis=1)

In [10]:
# 9. Transformar target a valor numerico mediante LabelEncoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [11]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_processed, y_train_encoded)

In [12]:
rf_model = RandomForestClassifier(
    n_estimators=100,           
    max_depth=13,              
    min_samples_split=5,        
    min_samples_leaf=3,         
    max_features=None,        
    bootstrap=True,           
    criterion='entropy',       
    class_weight='balanced',   
    random_state=42,
    n_jobs=-1                  
)

In [13]:
rf_model.fit(X_train_resampled, y_train_resampled)

y_pred = rf_model.predict(X_test_processed)

classes_x = y_pred.flatten().astype(int)

In [14]:
feature_importances = pd.DataFrame(
    rf_model.feature_importances_,
    index=X_train_processed.columns,
    columns=['Importance']
).sort_values('Importance', ascending=False)

print("\nTop 10 Most Important Features:")
print(feature_importances.head(10))


Top 10 Most Important Features:
                      Importance
FunctionalAssessment    0.191411
MMSE                    0.185176
ADL                     0.184381
MemoryComplaints_1      0.077988
BehavioralProblems_0    0.072829
MemoryComplaints_0      0.065816
BehavioralProblems_1    0.047766
DietQuality             0.018388
CholesterolHDL          0.015849
Age                     0.015274


In [15]:
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(len(classes_x)):
  if classes_x[i] == 1:
    if y_test_encoded[i] == 1:
      TP = TP + 1
    else :
      FP = FP + 1
  else:
    if y_test_encoded[i] == 0:
      TN = TN + 1
    else :
      FN = FN + 1

print('         ', 'label neg ', ' label pos')
print('pred neg    ', TN, "        ", FN)
print('pred pos    ', FP, "        ", TP)

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1 = 2*precision*recall / (precision + recall)
print("Precision:", precision)
print("Recall:", recall)
print("F1:", f1)

          label neg   label pos
pred neg     270          13
pred pos     7          140
Precision: 0.9523809523809523
Recall: 0.9150326797385621
F1: 0.9333333333333332
